# Week2_1 Assignment

# [BASIC](#Basic)
- BERT 모델의 hidden state에서 **특정 단어의 embedding을 여러 방식으로 추출 및 생성**할 수 있다.

# [CHALLENGE](#Challenge)
- **cosine similarity 함수를 구현**할 수 있다. 
- **단어들의 유사도**를 cosine similarity로 비교할 수 있다. 

# [ADVANCED](#Advanced)
- 문장 embedding을 구해 **문장 간 유사도**를 구할 수 있다.

### Reference
- [BERT word embedding & sentence embedding tutorial 영문 블로그](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#33-creating-word-and-sentence-vectors-from-hidden-states)

In [1]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

## Basic

### BERT 모델과 토크나이저 로드   
- 두 사람의 대화에서 (단어 및 문장의) embedding을 생성하고자 한다. 아래 대화를 BERT 모델에 입력해 출력값 중 "hidden states"값을 가져오자.
- `Hidden States`는 3차원 텐서를 가지고 있는 list 타입이다. List에는 BERT 모델의 각 layer마다의 hidden state 3차원 텐서를 갖고 있으며 각 텐서는 (batch_size, sequence_length, hidden_size) shape을 가진다. BERT-base 모델은 12 layer를 갖고 있고 이와 별도로 Embedding Layer 1개를 더 갖고 있기 때문에 `len(hidden states)`는 13개가 된다. 
    - batch_size: 학습 시 설정한 배치 사이즈. 또는 BERT 모델에 입력된 문장의 개수
    - sequence_length: 문장의 token의 개수. 
    - hidden size: token의 embedding size 
- Reference
    - [BertTokenizer.tokenize() 함수의 매개변수 설명](https://huggingface.co/transformers/v3.0.2/main_classes/tokenizer.html#transformers.PreTrainedTokenizer.__call__)
    - [BERTModel.forward() 함수의 매개변수 및 리턴 값 설명](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel.forward)

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 43.6 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 6.5 MB 32.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import BertTokenizer, BertModel

In [4]:
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-cased")
model_bert = BertModel.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
normal_person = ["what do you do when you have free time?"]
nerd = ["I code. code frees my minds, body and soul."]
normal_person.append("(what a nerd...) coding?")
nerd.append("Yes. coding is the best thing to do in the free time.")

for i in range(len(normal_person)):
    print(f"Normal Person asked: {normal_person[i]}")
    print(f"Nerd answers: {nerd[i]}")

Normal Person asked: what do you do when you have free time?
Nerd answers: I code. code frees my minds, body and soul.
Normal Person asked: (what a nerd...) coding?
Nerd answers: Yes. coding is the best thing to do in the free time.


In [6]:
# 매개변수 설명
# truncation <- max_len 넘어가지 않도록 자르기
# padding <- max(seq_len, max_len) zero padding
# return_tensors <- return 2d tensor 

inputs = tokenizer_bert(
    text = normal_person,
    text_pair = nerd,
    truncation = True,
    padding = "longest", 
    return_tensors='pt'
    )

print(inputs['input_ids'].shape)

torch.Size([2, 28])


In [7]:
# decoding
for i in range(len(inputs['input_ids'])):
    print(f"Coversation {i} -> '{tokenizer_bert.decode(inputs['input_ids'][i])}'")

Coversation 0 -> '[CLS] what do you do when you have free time? [SEP] I code. code frees my minds, body and soul. [SEP] [PAD] [PAD]'
Coversation 1 -> '[CLS] ( what a nerd... ) coding? [SEP] Yes. coding is the best thing to do in the free time. [SEP]'


In [8]:
# "code" 단어의 token id(각 단어에게 고유하게 주어진 id)를 출력
tokenizer_bert.encode('code', add_special_tokens=False)

[3463]

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [10]:
# 입력 데이터와 BERT 모델을 "GPU" 장치로 로드함
inputs = inputs.to(device)
model_bert.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [11]:
# 입력 데이터를 BERT 모델에 넣어 출력값을 가져옴
outputs = model_bert(
    **inputs, 
    output_hidden_states=True
    )

In [12]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [13]:
hidden_states = outputs['hidden_states']
print(f"# layers : {len(hidden_states)}")
print(f"tensor shape in each layer : {hidden_states[-1].shape}")

# layers : 13
tensor shape in each layer : torch.Size([2, 28, 768])


###  Q1. 1번째 sequence (문장)에서 "code"라는 단어의 인덱스를 모두 반환하라.
- "code" 단어는 총 2개 존재 

In [14]:
def get_index(seq, word):
  index = []
#   word = tokenizer_bert.encode(word, add_special_tokens=False)
  word = tokenizer_bert.convert_tokens_to_ids(word)
  for i, s in enumerate(seq):
      if s.detach().cpu().numpy() == word:
          index.append(i)
  return index


# input
# seq1: 1번째 sequence
# token: 단어
seq1 = inputs['input_ids'][0]
token = "code"

# output
token_index = get_index(seq1, token)
print(token_index)

[13, 15]


정답 코드

In [15]:
# code의 index 찾기
word = "code"
token_id = tokenizer_bert.encode(word, add_special_tokens=False)
print(token_id)
seq1 = inputs['input_ids'][0]
token_index = (seq1 == token_id[0]).nonzero()#[0]
print(token_index)

[3463]
tensor([[13],
        [15]], device='cuda:0')


In [27]:
(seq1 == token_id[0])

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False,  True, False, False, False, False,
        False, False, False, False, False, False, False, False],
       device='cuda:0')

In [28]:
(seq1 == token_id[0]).nonzero()

tensor([[13],
        [15]], device='cuda:0')

In [16]:
# validation
for index in token_index:
    assert word == tokenizer_bert.decode(seq1[index])

### Q2. 1번째 sequence의 1번째 "code" 토큰의 embedding을 여러가지 방식으로 구하고자 한다. BERT hidden state를 다음의 방식으로 인덱싱해 embedding을 구하라
- 1 layer
- last layer
- sum all 12 layers
- sum last 4 layers
- concat last 4 layers
- average last 4 layers


[concat last 4 layers 참고문헌](https://discuss.pytorch.kr/t/torchcat%EA%B3%BC-torchstack%EC%9D%80-%EC%96%B4%EB%96%BB%EA%B2%8C-%EB%8B%A4%EB%A5%B8%EA%B0%80%EC%9A%94/26)

내 코드

In [63]:
# 1 layer : squeeze() 해야함
first_layer_emb = hidden_states[1][0][token_index[0]]
print(first_layer_emb.shape)

# last layer : squeeze() 해야함
last_layer_emb = hidden_states[-1][0][token_index[0]]
print(last_layer_emb.shape)

# sum all 12 layers : 틀림
sum_all_layer_emb = sum(hidden_states[1:][0])[token_index[0]]
print(sum_all_layer_emb.shape)

# sum last 4 layers : 틀림
sum_last4_layer_emb = sum(hidden_states[-4:][0])[token_index[0]]
print(sum_last4_layer_emb.shape)

# concat last 4 layers : 틀림
concat_last4_layer_emb = torch.concat([hidden_states[-4][0], hidden_states[-3][0], hidden_states[-2][0], hidden_states[-1][0]], dim=1)[token_index[0], :]
print(concat_last4_layer_emb.shape)

# mean last 4 layers : 틀림
mean_last4_layer_emb = (sum(hidden_states[-4:][0])/4)[token_index[0]]
print(mean_last4_layer_emb.shape)

torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 3072])
torch.Size([1, 768])


정답 코드

In [68]:
index = token_index[0]

first_layer_emb = hidden_states[1][0, index.item(), :]
print(first_layer_emb.shape)

last_layer_emb = hidden_states[-1][0,  index.item(), :]
print(last_layer_emb.shape)

sum_all_layer_emb = sum([hs[0,  index.item(), :] for hs in hidden_states])
print(sum_all_layer_emb.shape)

sum_last4_layer_emb = sum([hidden_states[i][0, index.item(), :] for i in range(len(hidden_states)-1, len(hidden_states)-1-4, -1)])
print(sum_last4_layer_emb.shape)

concat_last4_layer_emb = torch.cat([hidden_states[i][0, index.item(), :] for i in range(len(hidden_states)-1, len(hidden_states)-1-4, -1)], dim=0)
print(concat_last4_layer_emb.shape)

mean_last4_layer_emb = sum_last4_layer_emb / 4
print(mean_last4_layer_emb.shape)

torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072])
torch.Size([768])


## Challenge

### Q3. `sum_last_four_layer` 방식으로 1번째 sequence의 2개의 "code" 토큰 사이의 코사인 유사도를 계산하라

내 코드 : 틀림

In [69]:
import torch.nn.functional as F

def cosine_similarity_manual(a, b, small_number=1e-8):
  # 영벡터 고려할 것
  result = F.cosine_similarity(x, y, dim=0, eps=small_number)
  return result

# input
# x: 1번째 sequence의 1번째 "code"의 sum_last_four_layer 방식 embedding
# y: 1번째 sequence의 2번째 "code"의 sum_last_four_layer 방식 embedding
x = sum(hidden_states[-4:][0])[token_index[0]]
y = sum(hidden_states[-4:][0])[token_index[1]]

# output
score = cosine_similarity_manual(x, y)
print(score)

tensor([ 1.0000,  1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000, -1.0000,
        -1.0000,  1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000, -1.0000,
         1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000,
        -1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000,
         1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000,  1.0000, -1.0000,
        -1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000, -1.0000,
         1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000,
         1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000, -1.0000,
        -1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000,  1.0000,
        -1.0000,  1.0000,  1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000,
        -1.0000, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000,  1.0000,  1.00

정답 코드

In [70]:
def cosine_similarity_manual(x,y,small_number=1e-8):

    def l2_norm(a):
        return torch.pow(a,2).sum().sqrt()
    
    return torch.inner(x,y) / max(l2_norm(x)*l2_norm(y), small_number)

In [71]:
sum_last4_layer_emb_1 = sum_last4_layer_emb
sum_last4_layer_emb_2 = sum([hidden_states[i][0, token_index[-1].item(), :] for i in range(len(hidden_states)-1, len(hidden_states)-1-4, -1)])
score = cosine_similarity_manual(sum_last4_layer_emb_1, sum_last4_layer_emb_2)
print(f"Similariy between first 'code' and second 'code' is {score}")

Similariy between first 'code' and second 'code' is 0.8426572680473328


### Q4. 2번째 sequence에서 "coding"이라는 토큰의 위치를 반환하라

내 코드

In [19]:
# Q1과 동일한 문제 

# input
# seq1: 2번째 sequence
# token: 단어
seq2 = inputs['input_ids'][1]
token = "coding"

# output
# Q1에서 구현한 함수 사용
token_index2 = get_index(seq2, token)
print(token_index2)

[10, 15]


정답 코드

In [72]:
# code의 index 찾기 
word = "coding"
token_id = tokenizer_bert.encode(word, add_special_tokens=False)
print(token_id)
seq2 = inputs['input_ids'][1]
token_index = (seq2 == token_id[0]).nonzero()#[0]
print(token_index)

[19350]
tensor([[10],
        [15]], device='cuda:0')


### Q5. `concat_last4_layer_emb` 방식으로 2번째 sequence의 2개의 "coding" 토큰 사이의 코사인 유사도를 계산하라

내 코드 : 틀림

In [73]:
# Q3과 동일한 문제

# input
# x: 2번째 sequence의 1번째 "coding"의 concat_last4_layer_emb
# y: 2번째 sequence의 2번째 "coding"의 concat_last4_layer_emb
x = torch.concat([hidden_states[-4][0], hidden_states[-3][0], hidden_states[-2][0], hidden_states[-1][0]], dim=1)[token_index2[0], :]
y = torch.concat([hidden_states[-4][0], hidden_states[-3][0], hidden_states[-2][0], hidden_states[-1][0]], dim=1)[token_index2[1], :]

# output
# Q3에서 구현한 함수 사용
score = cosine_similarity_manual(x, y)
print(score)

tensor(0.5848, device='cuda:0', grad_fn=<DivBackward0>)


In [74]:
x

tensor([ 0.4401, -1.0014,  0.5484,  ...,  0.0793, -0.4005,  0.3202],
       device='cuda:0', grad_fn=<SliceBackward0>)

정답 코드

In [75]:
concat_last4_layer_emb_1 = torch.cat([hidden_states[i][1, token_index[0].item(), :] for i in range(len(hidden_states)-1, len(hidden_states)-1-4, -1)], dim=0)
concat_last4_layer_emb_2 = torch.cat([hidden_states[i][1, token_index[-1].item(), :] for i in range(len(hidden_states)-1, len(hidden_states)-1-4, -1)], dim=0)
score =  cosine_similarity_manual(concat_last4_layer_emb_1, concat_last4_layer_emb_2)
print(f"Similariy between first '{word}' and second '{word}' is {score}")

Similariy between first 'coding' and second 'coding' is 0.8681785464286804


### Q6. 2번째 sequence에서 랜덤하게 토큰 하나를 뽑아보자. 그 랜덤 토큰과 2번째 sequence의 2번째 "coding" 토큰의 코사인 유사도를 계산해보자

내 코드 : 틀림

In [79]:
# input
# random_idx: random 모듈 사용하여 뽑은 랜덤 토큰의 인덱스
# random_word: random_idx에 해당하는 단어
# x: 2번째 sequence의 2번째 "coding" 토큰의 concat_last4_layer_emb
# y: 랜덤 토큰의 concat_last4_layer_emb

random_idx = random.randrange(0, len(inputs['input_ids'][1]))
random_word = inputs['input_ids'][1][random_idx]

x = torch.concat([hidden_states[-4][0], hidden_states[-3][0], hidden_states[-2][0], hidden_states[-1][0]], dim=1)[random_idx, :]
y = torch.concat([hidden_states[-4][0], hidden_states[-3][0], hidden_states[-2][0], hidden_states[-1][0]], dim=1)[token_index2[1], :]

# output
# Q3에서 구현한 함수 사용
score = cosine_similarity_manual(x, y)
print(score)

tensor(0.6831, device='cuda:0', grad_fn=<DivBackward0>)


In [80]:
random_idx

12

정답 코드

In [77]:
token_len = hidden_states[0][0].shape[0]
token_len

28

In [78]:
random_idx = random.randint(0,token_len-1)
print(random_idx)
random_token_id = inputs['input_ids'][-1][random_idx].item()
random_word = tokenizer_bert.decode([random_token_id])
print(f"Random word : {random_word}")

concat_last4_layer_emb_other = torch.cat([hidden_states[i][1, random_idx, :] for i in range(len(hidden_states)-1, len(hidden_states)-1-4, -1)], dim=0)
score =  cosine_similarity_manual(concat_last4_layer_emb_1, concat_last4_layer_emb_other)
print(f"Similariy between first '{word}' and second '{random_word}' is {score}")

6
Random word : .
Similariy between first 'coding' and second '.' is 0.5072759985923767


#### 질문할 것!
```
inputs['input_ids'][1][random_idx]의 ids에 해당하는 단어를 가져오는 방법
```


In [23]:
random_word

tensor(1110, device='cuda:0')

In [24]:
tokenizer_bert.decode(random_word, add_special_tokens=False)

'i s'

In [25]:
tokenizer_bert.convert_ids_to_tokens(random_word.item())

'is'

## Advanced

### Q7. 1번째 sequence와 2번째 sequence의 문장 유사도를 구해보자. 문장의 엠베딩은 마지막 레이어의 첫번째 토큰 ('[CLS]')으로 생성한다.

내 코드 : 정답

In [85]:
# input
# x: 1번째 sequence의 embedding
# y: 2번째 sequence의 embedding
x = hidden_states[-1][0][0]
y = hidden_states[-1][1][0]

# output
# Q3에서 구현한 함수 사용
score =  cosine_similarity_manual(x, y)
print(score)

tensor(0.8130, device='cuda:0', grad_fn=<DivBackward0>)


#### 질문

```
pooler_output과 last_hidden_state의 cls 토큰의 값이 다를 수 있나요?
```


정답 코드

In [83]:
sequence_1_embedding = hidden_states[-1][0, 0, :]
sequence_2_embedding = hidden_states[-1][1, 0, :]
print(f"Shape : {sequence_1_embedding.shape}")

Shape : torch.Size([768])


In [88]:
score = cosine_similarity_manual(sequence_1_embedding, sequence_2_embedding).item()
print(f"Similariy between the first sequence and the second sequence is {score}")

Similariy between the first sequence and the second sequence is 0.8130241632461548
